In [262]:
import build_train 
import pickle
import numpy as np
import tensorflow as tf
import parameters
from collections import defaultdict
from nltk.tokenize import TweetTokenizer
from nltk import tokenize
import re
import math
max_len = 10
#batch_size = 10000
hidden_size = 200
embedding_size = 128

In [263]:
def create_sent_corpus():
    line_split = []
    message_list = defaultdict(list)
    with open("train_labels_sample.txt","r") as fid:
            c = 0
            for line in fid:
                c+=1
                #if c > 100:
                #    break
    #             print("len", len(line))
                if len(line) > 1:
                    line_split = line.split(" ")
                    message_line = " ".join(line_split[2:])
                    #message_list.append(" ".join(line.split()[2:]))
                    message_list[line_split[0]].append((message_line,line_split[1]))
            #max_len = len(max(message_list, key=len))
            #print(message_list)
            print(" total ",c," lines read fron train_labels")
            return  message_list

In [287]:
def genTrainExamples(message_list, max_len, wrd2idx):
    feature_list = defaultdict(list)
    #found max_len greater that embedding size don't understand a thing about it. Keeping it 10 temporarily.
    c = 0
    sent_list = []
    tokenizer = TweetTokenizer(preserve_case=False)
    regex = re.compile(r'[\.\]\%\[\'",\?\*!\}\{<>\^-]')
    for key, message_line_list in message_list.items():
        #c += 1
        #if c > 70:
        #    break
        for sent,label in message_line_list:
            content = tokenizer.tokenize(sent)
            content = [word for word in content if not regex.match(word)]
            for w in content:                 
                if w in wrd2idx: #don't know if we have to take words comming only in wrd2idx
                    sent_list.append(wrd2idx[w])
            while len(sent_list) < max_len:
                sent_list.append(0)
            while len(sent_list) > max_len:
                sent_list.pop()
            assert len(sent_list) == 10
            feature_list[key].append((sent_list,label))
            sent_list = []  
    #print(feature_list)
    return feature_list
                

Try dense layer, try to figure out why reshape has to use 3*3, try to figure out drop out and also try to understand what happens if we give same heights to all filters, try to figure out the strides, also for varying heights try different height combinations. 

Used this remember to put this in refference if the structure eremains the same for cnn in future.
http://www.wildml.com/2015/12/implementing-a-cnn-for-text-classification-in-tensorflow/

In [389]:
class CNNModel(object):
     

    def __init__(self, graph, E, U):

        
        self.build_graph(graph, E, U)

    def build_graph(self, graph, E, U):
        """

        :param graph:
        :param embedding_array:
        :param Config:
        :return:
        """
        pooled_outputs = []
        with graph.as_default():
            # for CNN #####
            self.embeddings = tf.Variable(E, dtype=tf.float32)
            self.train_inputs = tf.placeholder(tf.int32, shape=[None,max_len])
            #self.train_inputs = tf.placeholder(tf.int32, shape=[batch_size])
            self.train_labels = tf.placeholder(tf.int32, shape=[None,1])
            embed = tf.nn.embedding_lookup(self.embeddings, self.train_inputs)
            print("embed after lookup",embed)
            embed = tf.expand_dims(embed, -1)
            total_h_grams = 3 * 3
            
            
            # for RNN ###
            self.user_embeddings = tf.Variable(U, dtype=tf.float32)
            self.user_id = tf.placeholder(tf.int32, shape=[1])
            
            user_embed = tf.nn.embedding_lookup(self.user_embeddings, self.user_id)
            print("user_embed after lookup",user_embed)
            hidden_layer_weights = tf.Variable(tf.random_normal([hidden_size, total_h_grams + embedding_size],
                                                                      stddev=1.0 / math.sqrt(hidden_size)))
            print("hidden layer weights",hidden_layer_weights)
            hidden_layer_bias = tf.Variable(tf.zeros([hidden_size,1]))
            
            print("hidden_layer_bias",hidden_layer_bias )
            output_layer_weights = tf.Variable(tf.random_normal([1 ,hidden_size ],
            
                                                                   stddev=1.0 / math.sqrt(hidden_size)))
            print("output_layer_weights",output_layer_weights)  
            output_layer_bias = tf.Variable(tf.zeros([1,1]))
            print("output_layer_bias",output_layer_bias)         
            
            # CNN #####
            for f_size in [1,3,5]:                
                filter_weights = tf.Variable(tf.truncated_normal([f_size, 128, 1, 3], stddev=0.1))
                bias_conv = tf.Variable(tf.zeros(3))
                conv = tf.nn.conv2d(embed,filter_weights,strides=[1, 1, 1, 1],padding="VALID")
                additive_bias = tf.nn.bias_add(conv, bias_conv)
                Relu_layer = tf.nn.relu(additive_bias)
                pooled = tf.nn.max_pool(Relu_layer,ksize=[1, max_len - f_size + 1, 1, 1],strides=[1, 1, 1, 1],padding='VALID')
                pooled_outputs.append(pooled)
            
            #print(pooled_outputs)
            
            concatenate_map_filters = tf.concat(pooled_outputs,3)
            print("Concatenated map filters", concatenate_map_filters)
            Cs_matrix = tf.reshape(concatenate_map_filters,[total_h_grams, -1])
            print("Cs_matrx",Cs_matrix)
            
            # RNN ###
            reshaped_user_embed = tf.reshape(user_embed,[embedding_size,-1])
            print("reshaped_user_embed", reshaped_user_embed)
            rnn_input = tf.concat([reshaped_user_embed,Cs_matrix],0)
            print("rnn_input",rnn_input)
            H_Cs_U = tf.matmul(hidden_layer_weights,rnn_input)
            print("H_Cs_U",H_Cs_U)
            hidden_layer_output = tf.add(H_Cs_U,hidden_layer_bias)
            print("hidden_layer_output",hidden_layer_output)
            activation_layer_output = tf.nn.relu(hidden_layer_output)
            print("activation_layer_output",activation_layer_output)
            self.prediction = tf.add(tf.matmul(output_layer_weights,activation_layer_output),output_layer_bias)
            print("prediction",self.prediction)
            self.cue_cnn_loss = tf.nn.softmax_cross_entropy_with_logits_v2(logits=self.prediction, labels=self.train_labels)
            
            optimizer = tf.train.GradientDescentOptimizer(1e-9)
            grads = optimizer.compute_gradients(self.cue_cnn_loss)
            clipped_grads = [(tf.clip_by_norm(grad, 5), var) for grad, var in grads]
            self.app = optimizer.apply_gradients(clipped_grads)
            
            
            self.init = tf.global_variables_initializer()
            
  
            
    def train(self, sess, trainFeats, max_len ):
        self.init.run()
        print("Initailized")
        c = 0
        max_num_steps = 10
        user_idx = {}      
        for user,value in trainFeats.items():
            
            #print(value)
            sent = []
            label = []
            try:
                user_id = user_idx[user]
            except KeyError:
                user_idx[user] = len(user_idx)
            u_id = user_idx[user]
            if u_id > 84: ################################### ADDED because of the error in training of user2vec
                break
            average_loss = 0
            for step in range(max_num_steps):
                
                for tuple_val in value:
                    sent_i , label_i = tuple_val[0], tuple_val[1]
                    sent.append(sent_i)
                    label.append([int(label_i)])
                
                #k=np.asarray(sent)
                #sent_t = np.transpose(k)
                #print("sent",sent)
                for i in range(len(label)):
                    #print("label",label[i])
                    #print("sent",sent[i])
                    feed_dict = {self.user_id.name:[u_id], self.train_inputs.name: [sent[i]],self.train_labels.name: [label[i]]}
                _, loss_val = sess.run([self.app, self.cue_cnn_loss], feed_dict=feed_dict)
                #print("type of loss val",loss_val)
                average_loss += loss_val[0]

                if step % 1 == 0:
                    if step > 0:
                        average_loss /= 1
                    print("Average loss at step ", step, ": ", average_loss)
                    average_loss = 0
            #print("next")
            c +=1
        print("train finished")
        #print(c)
        #print(len(self.pooled_outputs))
        #return self.pooled_outputs
           


In [390]:
def init_cnn():
    E,unigram_prob,wrd2idx,word_counter,n_users = pickle.load(open('train_embeddings.pkl', 'rb'))
    U = pickle.load(open("user_embeddings.pkl","rb"))
    print(E.shape)
    message_list = create_sent_corpus()
    print("Generating Traning Examples")
    trainFeats= genTrainExamples(message_list, max_len, wrd2idx)
    #print(trainFeats)
    print("Done.")

    # Build the graph model
    graph = tf.Graph()

    model = CNNModel(graph, E, U)

    with tf.Session(graph=graph) as sess:
        
        Cs_matrix = model.train(sess, trainFeats, max_len)
        
    

In [391]:
init_cnn()

(1152, 128)
 total  121  lines read fron train_labels
Generating Traning Examples
Done.
embed after lookup Tensor("embedding_lookup:0", shape=(?, 10, 128), dtype=float32)
user_embed after lookup Tensor("embedding_lookup_1:0", shape=(1, 128), dtype=float32)
hidden layer weights <tf.Variable 'Variable_2:0' shape=(200, 137) dtype=float32_ref>
hidden_layer_bias <tf.Variable 'Variable_3:0' shape=(200, 1) dtype=float32_ref>
output_layer_weights <tf.Variable 'Variable_4:0' shape=(1, 200) dtype=float32_ref>
output_layer_bias <tf.Variable 'Variable_5:0' shape=(1, 1) dtype=float32_ref>
Concatenated map filters Tensor("concat:0", shape=(?, 1, 1, 9), dtype=float32)
Cs_matrx Tensor("Reshape:0", shape=(9, ?), dtype=float32)
reshaped_user_embed Tensor("Reshape_1:0", shape=(128, 1), dtype=float32)
rnn_input Tensor("concat_1:0", shape=(137, 1), dtype=float32)
H_Cs_U Tensor("MatMul:0", shape=(200, 1), dtype=float32)
hidden_layer_output Tensor("Add:0", shape=(200, 1), dtype=float32)
activation_layer_outp

Average loss at step  7 :  0.0
Average loss at step  8 :  0.0
Average loss at step  9 :  0.0
Average loss at step  0 :  0.0
Average loss at step  1 :  0.0
Average loss at step  2 :  0.0
Average loss at step  3 :  0.0
Average loss at step  4 :  0.0
Average loss at step  5 :  0.0
Average loss at step  6 :  0.0
Average loss at step  7 :  0.0
Average loss at step  8 :  0.0
Average loss at step  9 :  0.0
Average loss at step  0 :  0.0
Average loss at step  1 :  0.0
Average loss at step  2 :  0.0
Average loss at step  3 :  0.0
Average loss at step  4 :  0.0
Average loss at step  5 :  0.0
Average loss at step  6 :  0.0
Average loss at step  7 :  0.0
Average loss at step  8 :  0.0
Average loss at step  9 :  0.0
Average loss at step  0 :  0.0
Average loss at step  1 :  0.0
Average loss at step  2 :  0.0
Average loss at step  3 :  0.0
Average loss at step  4 :  0.0
Average loss at step  5 :  0.0
Average loss at step  6 :  0.0
Average loss at step  7 :  0.0
Average loss at step  8 :  0.0
Average 

Average loss at step  0 :  0.0
Average loss at step  1 :  0.0
Average loss at step  2 :  0.0
Average loss at step  3 :  0.0
Average loss at step  4 :  0.0
Average loss at step  5 :  0.0
Average loss at step  6 :  0.0
Average loss at step  7 :  0.0
Average loss at step  8 :  0.0
Average loss at step  9 :  0.0
Average loss at step  0 :  0.0
Average loss at step  1 :  0.0
Average loss at step  2 :  0.0
Average loss at step  3 :  0.0
Average loss at step  4 :  0.0
Average loss at step  5 :  0.0
Average loss at step  6 :  0.0
Average loss at step  7 :  0.0
Average loss at step  8 :  0.0
Average loss at step  9 :  0.0
Average loss at step  0 :  0.0
Average loss at step  1 :  0.0
Average loss at step  2 :  0.0
Average loss at step  3 :  0.0
Average loss at step  4 :  0.0
Average loss at step  5 :  0.0
Average loss at step  6 :  0.0
Average loss at step  7 :  0.0
Average loss at step  8 :  0.0
Average loss at step  9 :  0.0
Average loss at step  0 :  0.0
Average loss at step  1 :  0.0
Average 